# 量子变分求解器在量子化学计算中的应用


`Linux` `CPU` `全流程` `初级` `中级` `高级`

[![](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/_static/logo_source.png)](https://gitee.com/mindspore/mindquantum/blob/master/tutorials/vqe_for_quantum_chemistry.ipynb)

## 概述

量子化学，指的是运用量子力学的基本理论及方法，求解含时或定态薛定谔方程的数值解。在高性能计算机上进行量子化学模拟已成为研究材料的物理、化学性质的重要手段。然而，精确求解薛定谔方程具有指数级的复杂度，可模拟的化学体系规模严重受制于此。近年量子计算的发展为解决这个问题提供了一条可行的路，有望在量子计算机上实现多项式复杂度下对薛定谔方程的高精度求解。

[Peruzzo等人](https://doi.org/10.1038/ncomms5213)在2014年首次将量子变分求解器(Variational quantum eigensolver, VQE)结合[幺正耦合簇理论](https://doi.org/10.1016/S0009-2614(89)87372-5)用于量子化学的模拟中，实现了He-H<sup>+</sup>基态能量的求解。量子变分求解器是一个量子-经典混合算法，在基于量子算法的化学模拟中应用广泛，本教程将介绍使用量子变分求解器求解分子体系基态能量的方法。

本教程的主要内容包括如下几个部分：

1. 量子化学原理简介
2. 量子变分求解器的应用
3. 使用mindquantum实现高效自动求导的VQE模拟

## 环境准备

本教程需要安装以下环境：
- NumPy
- SciPy
- [mindquantum](https://gitee.com/mindspore/mindquantum)
- [mindspore](https://gitee.com/mindspore/mindspore)
- PySCF
- openfermion
- openfermionpyscf

导入本教程所依赖模块

In [ ]:
import numpy as np
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import mindquantum as mq
from mindquantum import Circuit, X, RX, Hamiltonian
from mindquantum.circuit import generate_uccsd
from mindquantum.nn import generate_pqc_operator
import mindspore as ms
import mindspore.context as context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

## 量子化学计算方法

量子化学的核心问题在于求解薛定谔方程(Schrödinger Equation)。一般来说，求解含时薛定谔方程(Time-dependent Schrödinger Equation)较为复杂，故引入玻恩-奥本海默近似(Born-Oppenheimer approximation, BO approximation)。BO近似认为，原子核质量远大于电子、运动速度远低于电子，故可以将两者进行分离变量，单独讨论原子核或电子的运动，于是可得到如下不含时的电子运动方程，也称为定态薛定谔方程：

$$
\hat{H} |\Psi\rangle = E |\Psi\rangle
$$

其中$\hat{H}$包含以下三项：

$$
\hat{H} = \hat{K} _{e} + \hat{V} _{ee} + \hat{V} _{Ne}
$$

分别为电子动能、电子-电子势能和电子-核势能。

有多种数值算法可以求解定态薛定谔方程。本教程将介绍其中的一类：波函数方法。波函数方法直接求解给定分子哈密顿量的本征波函数和本征能量，目前有大量的开源软件包可实现，如[PySCF](http://pyscf.org/)等。此处从一个简单的例子：氢化锂分子开始，使用openfermion结合openfermionpyscf插件进行。首先定义分子结构：

In [2]:
dist = 1.5
geometry = [
    ["Li", [0.0, 0.0, 0.0 * dist]],
    ["H",  [0.0, 0.0, 1.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)

Geometry: 
 [['Li', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.5]]]


上面的代码定义了一个Li-H键长为1.5Å分子。使用STO-3G基组进行计算。接下来使用openfermionpyscf，调用PySCF进行HF、CCSD和FCI计算。这三种方法属于波函数方法，开始计算之前，先对这些方法作一个简单的介绍：

### 波函数方法

求解定态薛定谔方程的方法之一是[Hartree-Fock(HF)](https://doi.org/10.1098/rspa.1935.0085)方法，该方法在二十世纪三十年代左右由Hartree等人提出，是量子化学计算中的基本方法。HF方法引入了单行列式近似，即$N$-电子体系的波函数由一个行列式形式的波函数表示：

$$
| \Psi \rangle = | \psi_{1} \psi_{2} \psi_{3} \dots \psi_{N} \rangle
$$

其中$| \psi_{1} \psi_{2} \psi_{3} \dots \rangle$代表由一组自旋轨道波函数$\{ \pi_{i} \}$构成的N阶行列式。
自旋轨道波函数$\psi_{i}$可进一步用一组形式已知的基函数展开：

$$\psi_{i} = \phi_{i} \eta_{i}$$
$$\phi_{i} = \sum_{\mu}{C_{\mu i} \chi_{\mu}}$$

其中$\{\chi_{\mu}\}$被称为基函数，可以是高斯函数等。
该近似考虑了电子间的交换作用，但是忽略了电子间的关联作用，故无法正确计算如解离能等性质。

HF方法的改进可以从波函数展开定理出发。波函数展开定理可以表述为，若$\{ \psi_{i} \}$是一组完备的自旋轨道波函数，则$N$-电子体系波函数可以由$\{ \psi_{i} \}$构成的行列式波函数精确展开：

$$
| \Psi \rangle = \sum^{\infty} _ {i_{1} < i_{2} < \dots < i_{N}} {C_{i_{1} i_{2} \dots i_{N}} | \psi_{i_{1}} \psi_{i_{2}} \dots \psi_{i_{N}} \rangle}
$$

由此可得到Configuration Interaction(CI)方法：

$$
| \Psi_{CI} \rangle = C_{0} | \Psi_{HF} \rangle + \sum^{a\rightarrow\infty} _{i\in occ\\\\a\not\in occ}{C^{a} _{i} | \Psi^{a} _{i} \rangle } + \sum^{ab\rightarrow\infty} _{ij\in occ\\\\ab\not\in occ}{C^{ab} _{ij} | \Psi^{ab} _{ij} \rangle } 
$$

上式中的$| \Psi^{a}_{i} \rangle + \dots$代表电子由轨道$i$激发到轨道$a$的单激发波函数，以此类推。只考虑单激发和双激发的CI被称为CISD，即Configuration Interaction with singles and doubles。将基态HF波函数一直到N激发波函数全部考虑在内的Configuration Interaction被称为Full Configuration Interaction(FCI)，FCI波函数是定态薛定谔方程在给定基函数下的精确解。

### 二次量子化

在二次量子化表述下，体系的哈密顿量具有如下形式：

$$
\hat{H} = \sum_{p, q}{h^{p} _ {q} E^{p} _ {q}} + \sum_{p, q, r, s}{\frac{1}{2} g^{pq} _ {rs} E^{pq} _ {rs} }
$$

其中$E^{p} _ {q}$和$E^{pq} _ {rs}$分别为：

$$
E^{p} _ {q} = a^{\dagger} _ {p} a_{q}
$$
$$
E^{pq} _ {rs} = a^{\dagger} _ {p} a^{\dagger} _ {q} a_{r} a_{s}
$$

$a^{\dagger} _ {p}$和$a _ {q}$分别为产生算符(Creation Operator)和湮灭算符(Annihilation Operator)。

使用二次量子化的表述方法，可以非常方便地表示激发态波函数：

$$
| \Psi^{abc\dots} _ {ijk\dots} \rangle = a^{\dagger} _ {a} a^{\dagger} _ {b} a^{\dagger} _ {c} \dots a_{i} a_{j} a_{k} \dots | \Psi \rangle
$$

CI方法的一个改进是耦合簇理论(Coupled-Cluster theory, CC)。CC引入指数化算符：

$$
| \Psi_{CC} \rangle = \exp{(\hat{T})} | \Psi_{HF} \rangle
$$

其中耦合簇算符$\hat{T}$为对激发算符的求和：

$$
\hat{T} = \sum_{p\not\in occ\\\\q\in occ}{\theta^{p} _ {q} E^{p} _ {q}} + \sum_{pq\not\in occ\\\\rs\in occ}{\theta^{pq} _ {rs} E^{pq} _ {rs}} + \dots
$$

其中$\theta$和CI方法中的$C$类似，是待求解的参数。由指数的泰勒展开易知，即使耦合簇算符$\hat{T}$中只包含低阶激发项，$\exp{(\hat{T})}$也可以隐含部分高阶激发，这也使得CC方法向FCI波函数收敛的速度要远快于CI，同样截断到K激发，如K=2，CCSD的精度会超过CISD。


<!--
一般而言，若一个方法可以达到化学精度，即由此方法计算的能量和FCI能量之间的差值小于1 kcal/mol，则认为这个方法具有良好的精度，截断到三激发的CCSD(T)在大部分情况下都能符合这个标准
-->

电子关联作用的效果是使得总能量降低，故HF得到的基态能量会略高于CCSD和FCI。另外，从上述理论不难发现，FCI的计算量远大于CCSD和HF。我们使用openfermion封装的MolecularData和openfermionpyscf封装的run_pyscf函数来进行演示：

In [3]:
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -7.8633576215351200 Ha
CCSD energy:  -7.8823529091527051 Ha
FCI energy:  -7.8823622867987284 Ha


在上面的例子中，我们运行了Hartree-Fock(HF)、CCSD、FCI进行总能量的计算。若对运行时间进行统计，会发现$T_{HF}<T_{CCSD}\ll T_{FCI}$，换成计算量更大的体系如乙烯分子等会更明显一些。此外，对于计算得到的总能量，有$E_{HF}>E_{CCSD}>E_{FCI}$。计算完成后，我们将结果保存到`molecule_file`文件（即`molecule_of.filename`）中：

In [ ]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file)

量子化学计算的一大阻碍是计算量。随着体系大小（电子数、原子数）的增加，求解FCI波函数和基态能量的时间消耗大约以$2^{N}$增长，即使是较小的分子如乙烯分子等，进行FCI计算也并不容易。量子计算机的出现为此提供了一条可能的解决途径，已有的研究表明，量子计算机可以多项式的时间复杂度模拟哈密顿量的含时演化，在量子处理器上进行化学模拟相较于经典计算机有指数级的加速。本教程将介绍其中一类量子算法：量子变分求解器。

## 量子变分求解器

量子变分求解器(Variational Quantum Eigensolver, VQE)是一类量子-经典混合(Hybrid quantum-classical)算法，应用变分原理实现对基态波函数的求解。其中，变分参数的优化步在经典计算机上进行。

### 变分原理

变分原理可使用如下形式表述：

$$
E_{0} \le \frac{\langle \Psi_{t} | \hat{H} | \Psi_{t} \rangle}{\langle \Psi_{t} | \Psi_{t} \rangle}
$$

上式中的$| \Psi_{t} \rangle$代表试探波函数。变分原理表明，在满足一定的条件下，任意试探波函数得到的基态能量总是大于等于真实的基态能量。变分原理为求解分子基态薛定谔方程提供了一种方法：使用一个参数化的函数$f(\theta)$作为精确基态波函数的近似，通过优化参数$\theta$来逼近精确的基态能量。

### 初态制备

在二次量子化表述下，$N$-电子HF波函数也具有非常简洁的形式：

$$
| \Psi_{HF} \rangle = \prod^{i=0} _{N-1}{a^{\dagger} _{i}| 0 \rangle}
$$

上式搭建了一个由量子化学波函数到量子计算的桥梁：用$|0\rangle$代表非占据轨道，用$|1\rangle$代表电子占据的轨道，由此可以将$N$-电子HF波函数映射为由一串$M+N$个量子比特$| 00\dots 11\dots \rangle$，$M$代表非占据轨道的数量。

以下代码构造了对应于LiH分子的HF初态波函数。在Jordan-Wigner变换下，相当于将$N$个$\text{X}$门作用于$|000\dots\rangle$上。

In [5]:
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
print(hartreefock_wfn_circuit)

X(0)
X(1)
X(2)
X(3)


基于此，我们可以构造如下形式的试探波函数：

$$
| \Psi_{t} \rangle = U(\theta) | \Psi_{HF} \rangle
$$

其中$U(\theta)$代表一个可通过量子线路模拟的幺正变换，$| \Psi_{HF} \rangle$作为初态，可通过多个单比特$\text{X}$门来方便地制备。$U(\theta) | \Psi_{HF} \rangle$的具体形式也被称为波函数拟设。
### 波函数拟设

前文提到的耦合簇理论是一个非常高效的波函数拟设。在量子计算机上使用，需要作一些修改：

$$
| \Psi_{UCC} \rangle = \exp{(\hat{T} - \hat{T}^{\dagger})} | \Psi_{HF} \rangle
$$

UCC即幺正耦合簇(Unitary Coupled-Cluster theory)，$\hat{T}^{\dagger}$代表$\hat{T}$的厄米共轭。如此，$\exp{(\hat{T} - \hat{T}^{\dagger})}$即为幺正算符。[Peruzzo等人](https://doi.org/10.1038/ncomms5213)在2014年首次使用VQE结合UCCSD(Unitary coupled-cluster with singles and doubles)拟设进行了量子计算机上的化学模拟实验。值得注意的是幺正耦合簇默认了耦合簇算符中的参数$\{\theta\}$是实数。在分子体系中该假设不会有问题；在周期性体系中，[刘杰等人](https://doi.org/10.1021/acs.jctc.0c00881)的研究表明幺正耦合簇会因为忽略复数部分而造成误差。本教程暂时不讨论幺正耦合簇在周期性体系中的应用。

使用mindquantum的circuit模块中的`generate_uccsd`函数可读取先前保存在`molecule_file`的计算结果，“一键”构造UCCSD波函数拟设，以及其对应的量子线路：

In [6]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_file, th=-1)

ccsd:-7.882352909152705.
fci:-7.882362286798728.


`generate_uccsd`将幺正耦合簇相关的函数打包了起来，包括导出分子哈密度量、构造幺正耦合簇拟设算符、提取CCSD计算的耦合簇系数等多个步骤。在[分步构造幺正耦合簇拟设](#step-by-step)章节，我们会演示如何使用mindquantum的相关接口分步完成其中包含的步骤。完整的量子线路包含HF初态+UCCSD拟设，如下代码所示：

In [7]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

==============================Circuit Summary==============================
|Total number of gates  : 12612.                                          |
|Parameter gates        : 640.                                            |
|with 44 parameters are : p2, p4, p8, p37, p32, p31, p5, p14, p25, p12... |
|Number qubit of circuit: 12                                              |
Number of parameters: 44


对于LiH分子而言，其UCCSD波函数拟设中包含44个变分参数。该线路总共的量子比特门数量为12612，总共需要12个量子比特进行模拟。

### VQE的一般流程

使用VQE进行分子基态求解的一般流程如下：

1. 制备HF初态：$| 00\dots11\dots \rangle$；
2. 定义波函数拟设，如UCCSD等；
3. 将波函数拟设转化为参数化的量子线路；
4. 初始化变分参数，如全设为0等；
5. 在量子计算机上多次测量得到分子哈密顿量在该套变分参数下的能量$E(\theta)$以及能量关于参数的导数$\{ {\partial E} / {\partial \theta_{i}} \}$
6. 在经典计算机上使用优化算法，如梯度下降、BFGS等更新变分参数；
7. 将新的变分参数传入量子线路中进行更新；
8. 重复步骤(5)到(7)，直到满足收敛标准；
9. 结束

在第5步中，求取能量关于参数的导数$\{ {\partial E} / {\partial \theta_{i}} \}$在量子计算机上可通过parameter-shift rule来进行，在模拟器中也可通过模拟parameter-shift rule或者有限差分法来计算，是个较为耗时的过程。mindquantum基于mindspore框架，提供了类似于机器学习的自动求导功能，可以在模拟中可以高效计算参数化量子线路的导数。以下使用mindquantum构造带自动求导功能的参数化UCCSD量子线路：

In [8]:
molecule_pqc = generate_pqc_operator(
    ["null"], ansatz_parameter_names, 
    RX("null").on(0) + total_circuit, 
    Hamiltonian(hamiltonian_QubitOp))

由于mindquantum需要提供两套线路（以及参数）分别作为Encoding circuit和Ansatz circuit，此处我们使用`RX("null")`作为一个Encoding circuit，在之后令参数`null`等于0将其无效化。通过将参数的具体数值传入`molecule_pqc`，即可得到对应于此变分参数的能量$E(\theta)=\langle \Psi_{UCC}(\theta) | \hat{H} | \Psi_{UCC}(\theta) \rangle$以及关于每个变分参数的导数。

接下来需要进行VQE优化的(5)~(7)步，即对参数化量子线路进行优化。我们可以借助mindspore框架，使用参数化量子线路算子`molecule_pqc`构造一个机器学习模型，然后通过类似于训练神经网络的方法来优化变分参数：

In [9]:
class PQCNet(ms.nn.Cell):
    def __init__(self, pqc):
        super(PQCNet, self).__init__()
        self.pqc = pqc
        self.weight =  Parameter(initializer("Zeros",
            len(self.pqc.ansatz_params_names)),
            name="weight")
        self.encoder_data_dummy = ms.Tensor([[0]], 
            self.weight.dtype)

    def construct(self):
        energy, _, grads = self.pqc(self.encoder_data_dummy, self.weight)
        return energy

molecule_pqcnet = PQCNet(molecule_pqc)

此处我们手动构造了一个基本的`PQCNet`作为模型示例，该模型可以和常规的机器学习模型类似使用，比如优化权重、计算导数等。更好的选择是使用mindquantum中封装的`MindQuantumAnsatzOnlyLayer`，将会在后文中进行演示。

构造的`PQCNet`使用`"Zeros"`关键字，将所有的变分参数初始化为0。使用CCSD（耦合簇理论）或者MP2（二阶多体微扰论）的计算结果也可以作为幺正耦合簇变分参数的初始值。此时有$E(\vec{0})=\langle \Psi_{UCC}(\vec{0}) | \hat{H} | \Psi_{UCC}(\vec{0}) \rangle = E_{HF}$：

In [10]:
initial_energy = molecule_pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -7.8633575439453125


最后使用mindspore的Adam优化器进行优化，学习率设置为$1\times 10^{-2}$，优化终止标准设置为$\left.|\epsilon|\right. = \left.|E^{k+1} - E^{k}|\right. \le 1\times 10^{-8}$

In [11]:
optimizer = ms.nn.Adam(molecule_pqcnet.trainable_params(), learning_rate=1e-2)
train_pqcnet = ms.nn.TrainOneStepCell(molecule_pqcnet, optimizer)

eps = 1.e-8
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while (abs(energy_diff) > eps):
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 10 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", molecule_pqcnet.weight.asnumpy())

Step   0 energy  -7.8633575439453125
Step  10 energy  -7.8739027976989746
Step  20 energy  -7.8806061744689941
Step  30 energy  -7.8818173408508301
Step  40 energy  -7.8821077346801758
Step  50 energy  -7.8822517395019531
Step  60 energy  -7.8823161125183105
Step  70 energy  -7.8823409080505371
Optimization completed at step  71
Optimized energy:  -7.8823409080505371
Optimized amplitudes: 
 [ 3.7435061e-04  2.0003314e-03  3.4515299e-02  1.5759237e-02
  2.2138034e-04  8.8742212e-04  5.9856560e-05  1.4393384e-02
  6.4204316e-05  8.8728970e-04 -1.3264264e-04  1.4399975e-02
 -4.6512188e-04  5.3341745e-04  2.7783324e-03  5.2846380e-02
  1.2344022e-04 -2.0551911e-04  5.8422371e-05  6.4405147e-05
 -5.9763537e-05 -2.5284235e-04 -2.4150472e-04 -5.0080245e-05
 -6.0626553e-05 -1.9624823e-04  1.3753236e-04  9.2733640e-04
 -5.4294106e-02  3.0179413e-03  6.6223154e-05 -7.7102159e-05
  1.8220932e-04 -1.5686321e-04  6.5045337e-05  1.8604228e-06
  1.6173310e-04  1.6479529e-04  3.0232228e-03 -8.6284235e

可以看到，幺正耦合簇给出的计算结果和FCI非常接近，具有良好的精度。

## 分步构造幺正耦合簇拟设

<a id="step-by-step"></a>

在上文中，我们使用了`generate_uccsd`一步构造出了幺正耦合簇拟设所需要的所有内容，此处我们将步骤拆分，分别得到我们需要的耦合簇算符、对应的量子线路以及取自于经典CCSD计算结果的变分参数初猜值。
首先，导入部分额外依赖，主要包含mindquantum中hiqfermion模块的相关函数：

In [12]:
from mindquantum.hiqfermion.transforms import Transform
from mindquantum.hiqfermion.ucc import get_qubit_hamiltonian
from mindquantum.hiqfermion.ucc import uccsd_singlet_generator, uccsd_singlet_get_packed_amplitudes
from mindquantum.circuit import TimeEvolution
from mindquantum.nn.mindquantum_ansatz_only_layer import MindQuantumAnsatzOnlyLayer

分子哈密顿量使用`get_qubit_hamiltonian`，读取之前的计算结果得到：

In [13]:
hamiltonian_QubitOp = get_qubit_hamiltonian(molecule_of)

对于幺正耦合簇算符$ \hat{T} - \hat{T}^{\dagger} $，可以使用`uccsd_singlet_generator`进行构造。提供总量子比特数（总自旋轨道数）和总电子数，并设置参数`anti_hermitian=True`：

In [14]:
ucc_fermion_ops = uccsd_singlet_generator(
    molecule_of.n_qubits, molecule_of.n_electrons, anti_hermitian=True)

上一步构造的`ucc_fermion_ops`是参数化的。使用Jordan-Wigner变换将费米子激发算符映射为Pauli算符：

In [15]:
ucc_qubit_ops = Transform(ucc_fermion_ops).jordan_wigner()

接下来，我们需要得到幺正算符 $ \exp{(\hat{T} - \hat{T}^{\dagger})} $ 所对应的量子线路。`TimeEvolution`可生成$ \exp{(-i\hat{H}t)} $所对应的线路，其中$ \hat{H} $是一个厄米算符，$t$是实数。需要注意的是，使用`TimeEvolution`时，`ucc_qubit_ops`中已经包含了复数因子$i$，所以我们需要将`ucc_qubit_ops`除以$i$，或者提取其虚部：

In [16]:
ansatz_circuit = TimeEvolution(ucc_qubit_ops.imag, 1.0).circuit
ansatz_parameter_names = ansatz_circuit.para_name

我们使用`ansatz_parameter_names`记录该线路中的参数名。到目前为止，我们已经得到了VQE量子线路所需要内容，包括哈密顿量`hamiltonian_QubitOp`、参数化的波函数拟设线路`ansatz_circuit`，故可仿照前文，得到完整的态制备线路。其中Hartree-Fock参考态复用之前的`hartreefock_wfn_circuit`：

In [17]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()

=======================================Circuit Summary=======================================
|Total number of gates  : 12612.                                                            |
|Parameter gates        : 640.                                                              |
|with 44 parameters are : d2_26, d1_2, d2_23, d2_4, d2_6, d2_7, d2_17, d2_25, d2_3, d1_4... |
|Number qubit of circuit: 12                                                                |


下一步，需要为变分参数提供一个合理的初始值。前文构造的`PQCNet`默认使用0作为初猜，在大多数情况下是可行的。不过，使用CCSD的计算数据作为UCC的出发点，可能会有更好的结果。使用`uccsd_singlet_get_packed_amplitudes`函数从`molecule_of`提取CCSD的参数：

In [18]:
init_amplitudes_ccsd = uccsd_singlet_get_packed_amplitudes(
    molecule_of.ccsd_single_amps, molecule_of.ccsd_double_amps, molecule_of.n_qubits, molecule_of.n_electrons)
init_amplitudes_ccsd = [init_amplitudes_ccsd[param_i] for param_i in ansatz_parameter_names]

使用`MindQuantumAnsatzOnlyLayer`可以方便地由参数、量子线路获得以参数化量子线路为基础的机器学习模型：

In [19]:
molecule_pqcnet = MindQuantumAnsatzOnlyLayer(
    ansatz_parameter_names, total_circuit, Hamiltonian(hamiltonian_QubitOp.real))

使用`init_amplitudes_ccsd`（即CCSD计算的耦合簇系数）作为初始变分参数：

In [20]:
molecule_pqcnet.weight = Parameter(ms.Tensor(init_amplitudes_ccsd, molecule_pqcnet.weight.dtype))
initial_energy = molecule_pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -7.8173098564147949


在这个例子中，CCSD初猜并没有提供一个更好的起点。读者可以对更多的分子、更多种类的初始值（如随机数初猜）等进行测试和探究。最后进行VQE的优化步骤，优化器依然使用Adam，收敛标准不变。优化所用的代码与前文基本一致，注意更新相应的变量即可：

In [21]:
optimizer = ms.nn.Adam(molecule_pqcnet.trainable_params(), learning_rate=1e-2)
train_pqcnet = ms.nn.TrainOneStepCell(molecule_pqcnet, optimizer)

print("eps: ", eps)
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while (abs(energy_diff) > eps):
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 10 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", molecule_pqcnet.weight.asnumpy())

eps:  1e-08
Step   0 energy  -7.8173098564147949
Step  10 energy  -7.8745670318603516
Step  20 energy  -7.8764448165893555
Step  30 energy  -7.8813557624816895
Step  40 energy  -7.8817486763000488
Step  50 energy  -7.8822636604309082
Step  60 energy  -7.8822593688964844
Step  70 energy  -7.8823318481445312
Optimization completed at step  74
Optimized energy:  -7.8823318481445312
Optimized amplitudes: 
 [-3.55461583e-04  1.92621211e-03 -3.08955479e-02  1.54930232e-02
 -1.09486784e-04  1.03850279e-03  2.31159065e-04  1.41111370e-02
  1.21519442e-04  1.03762967e-03 -5.52369238e-05  1.41138518e-02
  3.43397754e-04  3.98736156e-04 -2.63991277e-03  5.25466762e-02
  1.26979663e-04 -1.63459510e-04  1.56383103e-04 -1.44725534e-04
  1.20656339e-04  1.42989345e-04 -1.40427408e-04 -1.22202100e-05
  6.13799330e-06  6.07158523e-04  2.66526640e-05 -3.88816370e-05
 -5.44567034e-02  3.19320243e-03 -5.67249253e-06  9.00171144e-05
 -7.03611586e-05  1.46694118e-04 -1.51760803e-04  1.52153356e-04
 -9.91804